In [20]:
from torch.utils.data import default_collate,DataLoader, Dataset
from torch import tensor as t, randn as r
import torch
from diy_stable_diffusion.training import Dataset
from diy_stable_diffusion.datasets import DataLoaders

In [15]:
latents = r((25, 1, 14, 14))
labels = torch.randint(0, 10, (25,))
latents.shape, labels.shape

(torch.Size([25, 1, 14, 14]), torch.Size([25]))

In [62]:
torch.lerp(torch.ones((3,)), torch.ones((3,)) * 10, 0.25)

tensor([3.2500, 3.2500, 3.2500])

In [68]:
'''
no args

batch is a list (len = batch size) of (x,y) tuples, in this case tuples (latent tsr, 0-dim label tsr)
'''
def my_collate(batch, *args, **kwargs):
    # print(f'batch type/len {type(batch), len(batch)}, args len {len(args)}')
    # print(f'batch 0th item type/len {type(batch[0]), len(batch[0])}')
    print(batch[0][0].shape)
    print(batch[0][1].item())
    return default_collate(batch)

def noisify(x0):
    dev = x0.device
    t = torch.rand((x0.shape[0],), device=dev).clamp(0,0.999)
    noise=torch.randn_like(x0)
    x_t = torch.lerp(x0, torch.randn_like(x0), 0.2)
    return (x_t, t,), noise


def noisify_collate_class(batch):
    collated = default_collate(batch)
    (x_t, t), eps = noisify(collated[0])
    c = collated[1]
    return (x_t, t, c), eps

dl= DataLoader(Dataset(latents, labels), batch_size=4, collate_fn=noisify_collate_class)

(xt, t, c), eps = next(iter(dl))
xt.shape, t.shape, c.shape, eps.shape

(torch.Size([4, 1, 14, 14]),
 torch.Size([4]),
 torch.Size([4]),
 torch.Size([4, 1, 14, 14]))

In [67]:
t, c

(tensor([0.6291, 0.9918, 0.4130, 0.8730]), tensor([6, 1, 7, 3]))